# Similarity Splitter 

In [1]:
import os,sys, tempfile, pandas as pd, shutil, logging, glob
sys.path.append("src")

In [2]:
base_data_dir = "tmp"

In [3]:
working_dir =  "tempspliiterhajkA"
if os.path.exists(working_dir) : shutil.rmtree(working_dir)
os.makedirs(working_dir, exist_ok=True)

In [4]:
def scores_to_df(scores, details, data_set, data_type, task_type):
    df = pd.DataFrame()
    for k , v in scores.items():
        df[k] = v
        
    for k , v in details.items():
        df[k + "_detail"] = v
    
    df["data_set"] = data_set + " " + data_type
    df["task_type"] = task_type
         
    return df

In [5]:
logging.basicConfig(level="WARN", handlers=[logging.StreamHandler(sys.stdout)],
                        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### 1. BC2GM

[Biocreative II gene mention](https://biocreative.bioinformatics.udel.edu/tasks/biocreative-ii/) overlap. Please download the test and train files for this task from the BioCreative Website.


In [6]:
bc2gm_train_file = os.path.join(base_data_dir, "train.in")
bc2gm_test_file = os.path.join(base_data_dir, "test.in")

bc2gm_train_eval_file = os.path.join(base_data_dir, "trainGENE.eval")
bc2gm_test_eval_file = os.path.join(base_data_dir, "testGENE.eval")

bc2gm_test_alteval_file = os.path.join(base_data_dir, "testALTGENE.eval")

# This is the predictions
bc2gm_predictions_file = os.path.join(base_data_dir, "bc2gm_result_test_pred.txt")

In [7]:
from bc2_gene_mention import BC2GeneMentionText 


bc2gmrun = BC2GeneMentionText()
thresholds = [0,25,50,75,100]
ngrams = [1]


### BC2GM : Threshold based simlarity

In [8]:

bc2gm_out_dir_text =  os.path.join(working_dir, "bc2gm_text")
os.makedirs(bc2gm_out_dir_text, exist_ok=True)
result_detail = bc2gmrun.run_similarity_threshold_splitter("text", bc2gm_train_file,  bc2gm_test_file, bc2gm_out_dir_text, bc2gm_test_eval_file, bc2gm_test_alteval_file,bc2gm_predictions_file ,thresholds,ngrams)
df_bc2_gm_text = pd.DataFrame(result_detail)
df_bc2_gm_text["dataset"] = "BC2GM (text)"

In [9]:
df_bc2_gm_text.head()

,ngram,min,max,num,percent,f-score,precision,recall,dataset
0,1,0,25,84,1.68,0.700000,0.538462,1.000000,BC2GM (text)
1,1,25,50,2910,58.20,0.816769,0.770923,0.868414,BC2GM (text)
2,1,50,75,1956,39.12,0.818354,0.780656,0.859877,BC2GM (text)
3,1,75,100,16,0.32,0.727273,0.800000,0.666667,BC2GM (text)
4,-1,0,100,5000,100.00,0.817055,0.774819,0.864160,BC2GM (text)


In [10]:
bc2gm_out_dir_anno =  os.path.join(working_dir, "bc2gm_text")
os.makedirs(bc2gm_out_dir_anno, exist_ok=True)
result_detail = bc2gmrun.run_similarity_threshold_splitter("eval", bc2gm_train_eval_file,  bc2gm_test_eval_file, bc2gm_out_dir_anno, bc2gm_test_eval_file, bc2gm_test_alteval_file, bc2gm_predictions_file ,thresholds,ngrams)
df_bc2_gm_anno = pd.DataFrame(result_detail)
df_bc2_gm_anno["dataset"] = "BC2GM (anno)"

In [11]:
df_bc2_gm_anno.head()

,ngram,min,max,num,percent,f-score,precision,recall,dataset
0,1,0,25,1056,16.679829,0.844543,0.843792,0.845296,BC2GM (anno)
1,1,25,50,396,6.254936,0.806846,0.797101,0.816832,BC2GM (anno)
2,1,50,75,1547,24.435318,0.834171,0.813953,0.855418,BC2GM (anno)
3,1,75,100,978,15.447797,0.863286,0.840774,0.887036,BC2GM (anno)
4,-1,0,100,6331,100.000000,0.817055,0.774819,0.864160,BC2GM (anno)


### BC2GM : Sorted based simlarity

In [12]:
num_parts = 4

In [13]:

bc2gm_out_dir_text =  os.path.join(working_dir, "bc2gm_text")
os.makedirs(bc2gm_out_dir_text, exist_ok=True)
result_score, result_detail = bc2gmrun.run_similarity_parts_splitter("text", bc2gm_train_file,  bc2gm_test_file, bc2gm_out_dir_text, bc2gm_test_eval_file, bc2gm_test_alteval_file,bc2gm_predictions_file ,num_parts)
df_bc2_gm_text_parts_score = pd.DataFrame(result_score)
df_bc2_gm_text_parts_score["dataset"] = "BC2GM (text)"

Exact matches Unigram, 39 / 5000
Exact matches Bigram, 26 / 5000
Exact matches Trigram, 26 / 5000


In [14]:
df_bc2_gm_text_parts_score.head()

,ngram,min,max,num,percent,f-score,precision,recall,dataset
0,1,0.000000,40.325254,1250,25.0,0.798133,0.741866,0.863636,BC2GM (text)
1,1,40.327957,47.140452,1250,25.0,0.820708,0.777716,0.868731,BC2GM (text)
2,1,47.140452,54.554473,1250,25.0,0.826751,0.790136,0.866924,BC2GM (text)
3,1,54.554473,100.000000,1250,25.0,0.816870,0.780074,0.857309,BC2GM (text)
4,-1,0.000000,100.000000,5000,100.0,0.817055,0.774819,0.864160,BC2GM (text)


## 2. SST2 Dataset

In [15]:
sst2_sentences_file = os.path.join("tmp", "datasetSentences.txt")
sst2_sentiment_labels_file  =  os.path.join("tmp", "sentiment_labels.txt")
sst2_dictionary_file  =  os.path.join("tmp", "dictionary.txt") 
sst2_datatset_split_file  =  os.path.join("tmp", "datasetSplit.txt") 



In [16]:
from sst2_dataset import SST2Dataset
SST2Dataset(sst2_sentences_file, sst2_sentiment_labels_file, sst2_datatset_split_file, sst2_dictionary_file)


2021-01-17 16:53:19,430 - sst2_dataset - WARNING - Text not found in dictionary: But in Imax 3-D , the clichÃ©s disappear into the vertiginous perspectives opened up by the photography .
2021-01-17 16:53:19,431 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- But it 's -RRB- worth recommending because of two marvelous performances by Michael Caine and Brendan Fraser .
2021-01-17 16:53:19,432 - sst2_dataset - WARNING - Text not found in dictionary: JirÃ­ Hubac 's script is a gem .
2021-01-17 16:53:19,433 - sst2_dataset - WARNING - Text not found in dictionary: You would n't call The Good Girl a date movie -LRB- an anti-date movie is more like it -RRB- , but when it 's good , it 's good and horrid .
2021-01-17 16:53:19,435 - sst2_dataset - WARNING - Text not found in dictionary: An incendiary , deeply thought-provoking look at one of the most peculiar -LRB- and peculiarly venomous -RRB- bigotries in our increasingly frightening theocracy
2021-01-17 16:53:19,438 - sst2_datas

2021-01-17 16:53:19,536 - sst2_dataset - WARNING - Text not found in dictionary: Dazzles with its fully-written characters , its determined stylishness -LRB- which always relates to characters and story -RRB- and Johnny Dankworth 's best soundtrack in years .
2021-01-17 16:53:19,537 - sst2_dataset - WARNING - Text not found in dictionary: The band 's courage in the face of official repression is inspiring , especially for aging hippies -LRB- this one included -RRB- .
2021-01-17 16:53:19,538 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Lawrence bounces -RRB- all over the stage , dancing , running , sweating , mopping his face and generally displaying the wacky talent that brought him fame in the first place .
2021-01-17 16:53:19,540 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- NÃ¦s -RRB- directed the stage version of Elling , and gets fine performances from his two leads who originated the characters on stage .
2021-01-17 16:53:19,542 - sst2_dataset -

2021-01-17 16:53:19,635 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Reno -RRB- delivers a monologue that manages to incorporate both the horror and the absurdity of the situation in a well-balanced fashion .
2021-01-17 16:53:19,635 - sst2_dataset - WARNING - Text not found in dictionary: ... the first 2\/3 of the film are incredibly captivating and insanely funny , thanks in part to interesting cinematic devices -LRB- cool visual backmasking -RRB- , a solid cast , and some wickedly sick and twisted humor ...
2021-01-17 16:53:19,639 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Haynes ' -RRB- homage to such films as `` All That Heaven Allows '' and `` Imitation of Life '' transcends them .
2021-01-17 16:53:19,643 - sst2_dataset - WARNING - Text not found in dictionary: SchÃ¼tte 's dramatic snapshot of the artist three days before his death offers an interesting bit of speculation as to the issues Brecht faced as his life drew to a close .
2021-01-17 1

2021-01-17 16:53:19,687 - sst2_dataset - WARNING - Text not found in dictionary: Lead actress GaÃ¯ , she of the impossibly long limbs and sweetly conspiratorial smile , is a towering siren .
2021-01-17 16:53:19,691 - sst2_dataset - WARNING - Text not found in dictionary: Even if you 've seen `` Stomp '' -LRB- the stage show -RRB- , you still have to see this !
2021-01-17 16:53:19,693 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- The digital effects -RRB- reminded me of Terry Gilliam 's rudimentary old Monty Python cartoons , in which he would cut out figures from drawings and photographs and paste them together .
2021-01-17 16:53:19,695 - sst2_dataset - WARNING - Text not found in dictionary: The fluid motion is astounding on any number of levels -- including the physical demands made on BÃ¼ttner -- and it implies in its wake the intractable , irreversible flow of history .
2021-01-17 16:53:19,696 - sst2_dataset - WARNING - Text not found in dictionary: This ready-made

2021-01-17 16:53:19,746 - sst2_dataset - WARNING - Text not found in dictionary: Jolting into Charleston rhythms , the story has the sizzle of old news that has finally found the right vent -LRB- accurate ?
2021-01-17 16:53:19,747 - sst2_dataset - WARNING - Text not found in dictionary: Who cares ? -RRB- .
2021-01-17 16:53:19,747 - sst2_dataset - WARNING - Text not found in dictionary: Damon brings the proper conviction to his role as -LRB- Jason Bourne -RRB- .
2021-01-17 16:53:19,748 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Barry -RRB- gives Assassin a disquieting authority .
2021-01-17 16:53:19,749 - sst2_dataset - WARNING - Text not found in dictionary: At its best -LRB- and it does have some very funny sequences -RRB- Looking for Leonard reminds you just how comically subversive silence can be .
2021-01-17 16:53:19,750 - sst2_dataset - WARNING - Text not found in dictionary: Transforms one of -LRB- Shakespeare 's -RRB- deepest tragedies into a smart new comedy

...

2021-01-17 16:53:20,122 - sst2_dataset - WARNING - Text not found in dictionary: Feels less like a change in -LRB- Herzog 's -RRB- personal policy than a half-hearted fluke .
2021-01-17 16:53:20,126 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Janey -RRB- forgets about her other obligations , leading to a tragedy which is somehow guessable from the first few minutes , maybe because it echoes the by now intolerable morbidity of so many recent movies .
2021-01-17 16:53:20,127 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Evans is -RRB- a fascinating character , and deserves a better vehicle than this facetious smirk of a movie .
2021-01-17 16:53:20,128 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- I -RRB- t 's certainly laudable that the movie deals with hot-button issues in a comedic context , but Barbershop is n't as funny as it should be .
2021-01-17 16:53:20,130 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Less a m

2021-01-17 16:53:20,178 - sst2_dataset - WARNING - Text not found in dictionary: With an unusual protagonist -LRB- a kilt-wearing Jackson -RRB- and subject matter , the improbable `` Formula 51 '' is somewhat entertaining , but it could have been much stronger .
2021-01-17 16:53:20,179 - sst2_dataset - WARNING - Text not found in dictionary: Sandra Bullock 's best dramatic performance to date -LRB- is -RRB- almost enough to lift -LRB- this -RRB- thrill-kill cat-and-mouser ... above its paint-by-numbers plot .
2021-01-17 16:53:20,180 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- The kid 's -RRB- just too bratty for sympathy , and as the film grows to its finale , his little changes ring hollow .
2021-01-17 16:53:20,181 - sst2_dataset - WARNING - Text not found in dictionary: Starts out ballsy and stylish but fails to keep it up and settles into clichÃ©s .
2021-01-17 16:53:20,182 - sst2_dataset - WARNING - Text not found in dictionary: Five screenwriters are credited wit

2021-01-17 16:53:20,233 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Creates -RRB- the worst kind of mythologizing , the kind that sacrifices real heroism and abject suffering for melodrama .
2021-01-17 16:53:20,235 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Allen 's -RRB- been making piffle for a long while , and Hollywood Ending may be his way of saying that piffle is all that the airhead movie business deserves from him right now .
2021-01-17 16:53:20,240 - sst2_dataset - WARNING - Text not found in dictionary: After the first 10 minutes , which is worth seeing , the movie sinks into an abyss of clichÃ©s , depression and bad alternative music .
2021-01-17 16:53:20,252 - sst2_dataset - WARNING - Text not found in dictionary: The film was produced by Jerry Bruckheimer and directed by Joel Schumacher , and reflects the worst of their shallow styles : wildly overproduced , inadequately motivated every step of the way and demographically targeted to 

2021-01-17 16:53:20,301 - sst2_dataset - WARNING - Text not found in dictionary: Return to Never Land is much more P.C. than the original version -LRB- no more racist portraits of Indians , for instance -RRB- , but the excitement is missing .
2021-01-17 16:53:20,302 - sst2_dataset - WARNING - Text not found in dictionary: By the end , you just do n't care whether that cold-hearted snake Petrovich -LRB- that would be Reno -RRB- gets his comeuppance .
2021-01-17 16:53:20,303 - sst2_dataset - WARNING - Text not found in dictionary: Don Michael Paul uses quick-cuts , -LRB- very -RRB- large shadows and wide-angle shots taken from a distance to hide the liberal use of a body double -LRB- for Seagal -RRB- .
2021-01-17 16:53:20,304 - sst2_dataset - WARNING - Text not found in dictionary: For something as splendid-looking as this particular film , the viewer expects something special but instead gets -LRB- sci-fi -RRB- rehash .
2021-01-17 16:53:20,305 - sst2_dataset - WARNING - Text not found i

2021-01-17 16:53:20,356 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- A -RRB- poorly executed comedy .
2021-01-17 16:53:20,358 - sst2_dataset - WARNING - A 569 out of 11855 were not found in dictionary


## 3. BC3 Article classification

In [17]:
bc3_act_train_file = os.path.join("tmp", "bc3_act_all_records.tsv")
bc3_act_test_file = os.path.join("tmp", "bc3_act_all_records_test.tsv")

In [18]:
from bc3_article_classification import BC3ArticleClassification

BC3ArticleClassification()


## Summary

In [19]:
formatters = { t:lambda x: "{:.2%}".format(x).replace("%","")  for t in {"f-score", "precision", "recall"}}
float_formatter = lambda x: "{:.2f}".format(x)

### Summary Threshold

In [20]:
df_threshold_summary = pd.concat([df_bc2_gm_text])

In [21]:
df_threshold_summary

,ngram,min,max,num,percent,f-score,precision,recall,dataset
0,1,0,25,84,1.68,0.700000,0.538462,1.000000,BC2GM (text)
1,1,25,50,2910,58.20,0.816769,0.770923,0.868414,BC2GM (text)
2,1,50,75,1956,39.12,0.818354,0.780656,0.859877,BC2GM (text)
3,1,75,100,16,0.32,0.727273,0.800000,0.666667,BC2GM (text)
4,-1,0,100,5000,100.00,0.817055,0.774819,0.864160,BC2GM (text)


In [22]:
print(df_threshold_summary.to_latex(float_format=float_formatter, formatters=formatters))

\begin{tabular}{lrrrrrrrrl}
\toprule
{} &  ngram &  min &  max &   num &  percent & f-score & precision & recall &       dataset \\
\midrule
0 &      1 &    0 &   25 &    84 &     1.68 &   70.00 &     53.85 & 100.00 &  BC2GM (text) \\
1 &      1 &   25 &   50 &  2910 &    58.20 &   81.68 &     77.09 &  86.84 &  BC2GM (text) \\
2 &      1 &   50 &   75 &  1956 &    39.12 &   81.84 &     78.07 &  85.99 &  BC2GM (text) \\
3 &      1 &   75 &  100 &    16 &     0.32 &   72.73 &     80.00 &  66.67 &  BC2GM (text) \\
4 &     -1 &    0 &  100 &  5000 &   100.00 &   81.71 &     77.48 &  86.42 &  BC2GM (text) \\
\bottomrule
\end{tabular}



### Summary Split by quartile

In [23]:
df_quartile_summary = pd.concat([df_bc2_gm_text_parts_score])

In [24]:
df_quartile_summary

,ngram,min,max,num,percent,f-score,precision,recall,dataset
0,1,0.000000,40.325254,1250,25.0,0.798133,0.741866,0.863636,BC2GM (text)
1,1,40.327957,47.140452,1250,25.0,0.820708,0.777716,0.868731,BC2GM (text)
2,1,47.140452,54.554473,1250,25.0,0.826751,0.790136,0.866924,BC2GM (text)
3,1,54.554473,100.000000,1250,25.0,0.816870,0.780074,0.857309,BC2GM (text)
4,-1,0.000000,100.000000,5000,100.0,0.817055,0.774819,0.864160,BC2GM (text)


In [25]:
print(df_quartile_summary.to_latex( float_format=float_formatter, formatters=formatters))

\begin{tabular}{lrrrrrrrrl}
\toprule
{} &  ngram &   min &    max &   num &  percent & f-score & precision & recall &       dataset \\
\midrule
0 &      1 &  0.00 &  40.33 &  1250 &    25.00 &   79.81 &     74.19 &  86.36 &  BC2GM (text) \\
1 &      1 & 40.33 &  47.14 &  1250 &    25.00 &   82.07 &     77.77 &  86.87 &  BC2GM (text) \\
2 &      1 & 47.14 &  54.55 &  1250 &    25.00 &   82.68 &     79.01 &  86.69 &  BC2GM (text) \\
3 &      1 & 54.55 & 100.00 &  1250 &    25.00 &   81.69 &     78.01 &  85.73 &  BC2GM (text) \\
4 &     -1 &  0.00 & 100.00 &  5000 &   100.00 &   81.71 &     77.48 &  86.42 &  BC2GM (text) \\
\bottomrule
\end{tabular}

